In [2]:
!pip install googletrans
!pip install mtranslate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=4a23bc16c73f98d96ca0b6d5a9363807de6cecf7aeb15b876d7f2bc67ec53c99
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling char

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from googletrans import Translator
import numpy as np

# Cargar el tokenizador BERT pre-entrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Cargar el modelo BERT pre-entrenado para clasificación de secuencias
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

# Establecer el modo de evaluación para el modelo
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [34]:
import pandas as pd
import mtranslate as mt

# Función para detectar y traducir idiomas utilizando BERT y Google Translate
def detect_and_translate_languages_with_bert(languages):
    translated_languages = []
    translator = Translator()
    languagesList = eval(languages)

    print(languagesList)

    for lang in languagesList:
        # Tokenizar la frase con BERT
        inputs = tokenizer(lang, return_tensors="pt", max_length=128, truncation=True)

        # Obtener la salida del modelo BERT
        with torch.no_grad():
            outputs = model(**inputs)

        # Obtener la predicción del idioma utilizando la salida del modelo
        predicted_language_index = np.argmax(outputs.logits.numpy())
        predicted_language = 'english' if predicted_language_index == 0 else 'non-english'

        if lang == "English":
            translated_languages.append(lang)
        else:
            print(lang)
            try:
                translated_lang = mt.translate(lang, 'en', 'auto')
                print(f"lang: {lang} translated: {translated_lang}")
                translated_languages.append(translated_lang)
            except Exception as e:
                print(f"Error occurred: {e}")
                translated_languages.append(lang)

    return translated_languages

movies_data = pd.read_csv("movies_new_notna.csv")
# Aplicar la función a cada fila de la columna "languages_availables"
movies_data['languages_availables'] = movies_data['languages_availables'].head(5000).apply(detect_and_translate_languages_with_bert)
movies_data.head()

Streaming output truncated to the last 5000 lines.
['English']
['English']
['English']
['English']
['English', 'Français']
Français
lang: Français translated: French
['English']
['English']
['हिन्दी', '', 'Français']
हिन्दी
lang: हिन्दी translated: Hindi

lang:  translated: 
Français
lang: Français translated: French
['English', 'Français']
Français
lang: Français translated: French
['English']
['English']
['Magyar', 'Pусский']
Magyar
lang: Magyar translated: Hungarian
Pусский
lang: Pусский translated: Russian
['English', 'Português']
Português
lang: Português translated: Portuguese
['English', 'Magyar']
Magyar
lang: Magyar translated: Hungarian
['广州话 / 廣州話', 'English']
广州话 / 廣州話
lang: 广州话 / 廣州話 translated: Cantonese / Cantonese
['广州话 / 廣州話', 'English']
广州话 / 廣州話
lang: 广州话 / 廣州話 translated: Cantonese / Cantonese
['English']
['English']
['English']
['English']
['English']
['English']
['English']
['English']
['English']
['广州话 / 廣州話', 'English', '普通话']
广州话 / 廣州話
lang: 广州话 / 廣州話 translated

,title,genres_associated,languages_availables,runtime,overview,adult,release_date,tagline,vote_average,keywords,cast
0,Jumanji,"['Adventure', 'Fantasy', 'Family']","[English, French]",104.0,When siblings Judy and Peter discover an encha...,False,1995-12-15,Roll the dice and unleash the excitement!,6.9,"['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
1,Grumpier Old Men,"['Romance', 'Comedy']",[English],101.0,A family wedding reignites the ancient feud be...,False,1995-12-22,Still Yelling. Still Fighting. Still Ready for...,6.5,"['fishing', 'best friend', 'duringcreditssting...","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret..."
2,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",[English],127.0,"Cheated on, mistreated and stepped on, the wom...",False,1995-12-22,Friends are the people who let you be yourself...,6.1,"['based on novel', 'interracial relationship',...","['Whitney Houston', 'Angela Bassett', 'Loretta..."
3,Father of the Bride Part II,['Comedy'],[English],106.0,Just when George Banks has recovered from his ...,False,1995-02-10,Just When His World Is Back To Normal... He's ...,5.7,"['baby', 'midlife crisis', 'confidence', 'agin...","['Steve Martin', 'Diane Keaton', 'Martin Short..."
4,Heat,"['Action', 'Crime', 'Drama', 'Thriller']","[English, Español]",170.0,"Obsessive master thief, Neil McCauley leads a ...",False,1995-12-15,A Los Angeles Crime Saga,7.7,"['robbery', 'detective', 'bank', 'obsession', ...","['Al Pacino', 'Robert De Niro', 'Val Kilmer', ..."


In [39]:
def process_languages(languages_list):
    if isinstance(languages_list, list):
        if 'Español' in languages_list:
            # Copiar los idiomas que tenga
            new_languages = languages_list[:]
            # Eliminar 'Español'
            new_languages.remove('Español')
            # Agregar 'Spanish'
            new_languages.append('Spanish')
            return new_languages
        else:
            return languages_list
    else:
        return ['English']

movies_data['languages_availables'] = movies_data['languages_availables'].apply(process_languages)

print(movies_data)

                                  title  \
0                               Jumanji   
1                      Grumpier Old Men   
2                     Waiting to Exhale   
3           Father of the Bride Part II   
4                                  Heat   
...                                 ...   
20644  Pooh's Heffalump Halloween Movie   
20645                 The Morning After   
20646                  House of Horrors   
20647               The Burkittsville 7   
20648                          Betrayal   

                              genres_associated languages_availables  runtime  \
0            ['Adventure', 'Fantasy', 'Family']    [English, French]    104.0   
1                         ['Romance', 'Comedy']            [English]    101.0   
2                ['Comedy', 'Drama', 'Romance']            [English]    127.0   
3                                    ['Comedy']            [English]    106.0   
4      ['Action', 'Crime', 'Drama', 'Thriller']   [English, Spanish]    170.0 

In [42]:
empty_lists_count = movies_data['languages_availables'].apply(lambda x: isinstance(x, list) and len(x) == 0).sum()

print("Número de filas con una lista vacía en 'languages_availables':", empty_lists_count)

Número de filas con una lista vacía en 'languages_availables': 0


In [43]:
# Guardar el DataFrame actualizado
from google.colab import drive
drive.mount('/content/drive')

movies_data.to_csv("/content/drive/MyDrive/dataIA/movies/movies_with_languagesRevised.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
